In [4]:
import pandas as pd
import numpy as np
from os import path as pth

In [5]:
path = r'/Users/polusa/Library/Mobile Documents/com~apple~CloudDocs/my_DA_2024/CareerFoundry_Data_Analytics_Bootcamp/4-Python_Fundamentals_for_DA/04-2024_Instacart_Basket_Analysis/02-Data'
prepared_data_folder = r'02-Prepared_Data'
raw_data_folder = r'01-Raw_data'

In [6]:
df_prods = pd.read_csv(pth.join(path, prepared_data_folder, 'products_checked.csv'), index_col=[0])
df_ords = pd.read_csv(pth.join(path, prepared_data_folder, 'orders_checked.csv'), index_col=[0])

In [7]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [8]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,-1.0
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


You’ve probably noticed by now that your df_ords and df_prods dataframes don’t contain a common column. How, then, are you supposed to combine them?

In [9]:
print(f'orders columns: {df_ords.columns},\n\n\
products columns: {df_prods.columns}')

orders columns: Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_last_order'],
      dtype='object'),

products columns: Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')


We will combine the `df_ords` dataframe with a new dataframe called `orders_products_prior`.  

This dataframe contains a `product_id` column, the same as your `df_prods` dataframe. By adding this column to your `df_ords` dataframe, you’ll have created a common column between `df_ords` and `df_prods`.

In [10]:
df_ords_prior = pd.read_csv(pth.join(path, raw_data_folder, 'orders_products_prior.csv'), index_col=False)

In [11]:
df_ords_prior.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [12]:
df_ords_prior.shape

(32434489, 4)

In [13]:
# merge using a inner join (default option for the how argument)
df_merged_large = pd.merge(df_ords_prior, df_ords, on = 'order_id', indicator=True)

In [14]:
df_merged_large.shape

(32434489, 10)

Let’s see what the merge flag frequency shows you here. In the `_merge` column, you can see that there are only entries that have a value of “both,” leading you to think that your key column, `order_id` exists completely in both dataframes. __However, this conclusion is wrong__.  

What pandas does here is fill in information about each product for every `order_id` in the `df_ords` dataframe.  

But does this mean that you have a full match? __The answer is no__. There’s one particular intricacy when using and interpreting the merge flag, and it has a lot to do with the way you chose to merge the dataframes. In this case, you chose the default option of inner join. This means that the resulting table will only contain observations found in both dataframes. As such, the merge flag here will only show entries that have a value of “both”.



In [15]:
# verify if it was a full match
df_merged_large['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [16]:
df_merged_large.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,_merge
0,2,33120,1,1,202279,3,5,9,8.0,both
1,2,28985,2,1,202279,3,5,9,8.0,both
2,2,9327,3,0,202279,3,5,9,8.0,both
3,2,45918,4,1,202279,3,5,9,8.0,both
4,2,30035,5,0,202279,3,5,9,8.0,both


Let's see what happened if we merge using an outer join:

In [17]:
merge_test = pd.merge(df_ords_prior, df_ords, on = 'order_id', how ='outer', indicator=True)

In [18]:
merge_test

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,_merge
0,1,NaN,NaN,NaN,112108,4,4,10,9.0,right_only
1,2,33120.0,1.0,1.0,202279,3,5,9,8.0,both
2,2,28985.0,2.0,1.0,202279,3,5,9,8.0,both
3,2,9327.0,3.0,0.0,202279,3,5,9,8.0,both
4,2,45918.0,4.0,1.0,202279,3,5,9,8.0,both
...,...,...,...,...,...,...,...,...,...,...
32640693,3421083,39678.0,6.0,1.0,25247,24,2,6,21.0,both
32640694,3421083,11352.0,7.0,0.0,25247,24,2,6,21.0,both
32640695,3421083,4600.0,8.0,0.0,25247,24,2,6,21.0,both
32640696,3421083,24852.0,9.0,1.0,25247,24,2,6,21.0,both


In [19]:
merge_test['_merge'].value_counts()

_merge
both          32434489
right_only      206209
left_only            0
Name: count, dtype: int64

When you perform an outer merge using `pd.merge()` with the `how='outer'` parameter, it combines the DataFrames `df_ords_prior` and `df_ords` based on the `order_id` column. Here's what each part of the merge represents:

Left DataFrame (`df_ords_prior`): This contains all the rows from df_ords_prior.  
Right DataFrame (df_ords): This contains all the rows from df_ords.  
Outer Join: This includes all rows from both DataFrames, filling in missing values with NaN (Not a Number) where there are no matches.  

Now, when you have 206,209 rows in the "right only" DataFrame, it means that there are rows in `df_ords` (the right DataFrame) that do not have corresponding matches in `df_ords_prior` (the left DataFrame).  
These are the rows that are exclusive to the right DataFrame after the merge.  

__After using this method to double-check your merge, you can see that you don’t actually have a full match__.  

You should always double-check your merge using an outer join, as well, especially when you’re exploring new data and performing test merges.

### Exporting Data in Pickle Format

A __pickle__, or “.pkl,” is a pandas format used to store data on your computer. While it’s similar to “.csv” files, pickles can only be opened using Python. Importing a pickle into your Jupyter notebook follows the same procedure as importing a “.csv” file and produces the same dataframe.

#### CSV Files
##### Advantages:

- Can be opened in multiple tools and programs (Excel, SAS, R)
- Can be customized to include certain columns or rows when imported
- Have a high compression rate when zipped
##### Disadvantages:

- Take more time to import and export when data sets are large
- Can lead to index column issues when exporting and reimporting

#### PKL Files
##### Advantages:

- Can be imported and exported quickly
- Save dataframes exactly as they look in Jupyter, guaranteeing your reimported dataframe won’t be changed
- Have a high compression rate when zipped
##### Disadvantages:

- Are only accessible to Python users
- Can’t be customized to include certain columns or rows when imported

Your `df_merged_large dataframe`, for instance, would likely take around two minutes to export as a pickle, while it could take upwards of ten minutes to export as a “.csv” file.

In [20]:
# export in csv
df_merged_large.to_csv(pth.join(path, prepared_data_folder, 'orders_products_combined.csv'))

In [21]:
# export in pkl
df_merged_large.to_pickle(pth.join(path, prepared_data_folder, 'orders_products_combined.pkl'))